In [27]:
import os
import json
import psycopg2

from dotenv import load_dotenv
from urllib.parse import quote_plus
from dune_client.client import DuneClient

load_dotenv()
dune = DuneClient.from_env()

# Define the additional option with URL encoding
encoded_option = quote_plus(f"endpoint={os.getenv('DATABASE_ENDPOINT')}")

# Connect using psycopg2 with connection string
connection_string = f"postgresql://{os.getenv('DATABASE_USER')}:{os.getenv('DATABASE_PASSWORD')}@{os.getenv('DATABASE_HOST')}:{os.getenv('DATABASE_PORT')}/{os.getenv('DATABASE_NAME')}?options={encoded_option}"
conn = psycopg2.connect(connection_string)
cur = conn.cursor()

# Sample database operation: fetching the current date and time from the server
cur.execute("SELECT NOW();")
current_time = cur.fetchone()
print(f"Current database server time: {current_time[0].strftime('%Y-%m-%d %H:%M')} UTC")

Current database server time: 2024-09-05 15:14 UTC


In [2]:
# Get most recent cat in database
cur.execute("""SELECT cat_number FROM public.cats ORDER BY cat_number DESC LIMIT 1""")
most_recent_cat = cur.fetchall()[0][0]
print(f"Database is up to date up until cat {most_recent_cat}")

Database is up to date up until cat 62719


In [3]:
# This does NOT execute the query, only fetches most recent cached result
query_result = dune.get_latest_result(3837577)

In [4]:
res = query_result.get_rows()

In [5]:
len(res)

1000

In [ ]:
res[0]

In [7]:
insert_query = """
INSERT INTO public.cats (
  cat_number,
  lock_time,
  block_height,
  minted_at,
  minted_by,
  feerate,
  tx_hash,
  tx_input,
  tx_input_count,
  tx_output,
  tx_output_count,
  tx_fee,
  tx_virtual_size
) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [8]:
for row in res:
    # TODO: Handle cases with OP_RETURN as first output, use tx_output[1][1][0] as minter for them

    if row["cat_number"] > most_recent_cat:
        # Construct INSERT for the normal case
        cur.execute(insert_query, (
            row["cat_number"],
            row["lock_time"],
            row["block_height"],
            row["block_time"],
            row["output"][0][1][0],
            row["feeRate"],
            row["id"][2:],
            json.dumps(row["input"]),
            len(row["input"]),
            json.dumps(row["output"]),
            len(row["output"]),
            row["fee"],
            row["virtual_size"],
        ))
conn.commit()

## Get block_hash and update table

In [9]:
# block_height -> block_hash
query_result = dune.get_latest_result(3837893)

In [10]:
res = query_result.get_rows()

In [11]:
len(res)

1805

In [12]:
res[0]

{'block_hash': '0x000000000000000000018e3ea447b11385e3330348010e1b2418d0d8ae4e0ac7',
 'block_height': 824205}

In [25]:
# Get most recent row with block_hash in database
cur.execute("""SELECT block_height FROM public.cats WHERE block_hash != '' ORDER BY cat_number DESC LIMIT 1""")
most_recent_block = cur.fetchall()[0][0]
print(f"Database is up to date up until block {most_recent_block}")

Database is up to date up until block 859854


In [28]:
for row in res:
    block_hash_without_prefix = row["block_hash"][2:]
    #if row["block_height"] <= most_recent_block:
    #    continue
    cur.execute(f"UPDATE public.cats SET block_hash='{block_hash_without_prefix}' WHERE block_height={row['block_height']}")
conn.commit()

# Create table definition

Only included here for reference, not necessary to run again unless bootstrapping a new database from scratch.